In [ ]:
import os

from fnmatch import fnmatch
from pathlib import PurePath

In [ ]:
EXCLUDES = {
    ".git",
    ".github",
    ".vscode",
    "build",
    "dist",
    "lib",
    "node_modules",
    "__pycache__",
    ".ipynb_checkpoints"
}

In [ ]:
def collect_files(root_path='..'):
    files = []
    for dirpath, dirnames, filenames in os.walk(root_path, followlinks=True):
        dirnames[:] = [d for d in dirnames if d not in EXCLUDES]
        for f in filenames:
            fullpath = PurePath(dirpath).relative_to(root_path).joinpath(f)

            if fullpath.parts not in files:
                files.append(fullpath.parts)
    files.sort()
    return files

In [ ]:
ls = collect_files()

In [ ]:
ls[:10]

In [ ]:
tree = {}
for f in ls:
    node = tree
    for part in f:
        if part not in node:
            node[part] = {}
        node = node[part]

In [ ]:
from ipytree import Node, Tree

In [ ]:
def create_tree_widget(root, path, depth=0):
    node = Tree() if depth == 0 else Node()
    for name, children in root.items():
        fullpath = path + [name]
        if len(children) == 0:
            leaf = Node(name)
            leaf.icon = 'file'
            leaf.icon_style = 'warning'
            node.add_node(leaf)
        else:
            subtree = create_tree_widget(children, fullpath, depth + 1)
            subtree.icon = 'folder'
            subtree.icon_style = 'info'
            subtree.name = name
            node.add_node(subtree)
    return node

In [ ]:
file_tree = create_tree_widget(tree, [])

In [ ]:
from ipylab import JupyterFrontEnd, Panel
from ipywidgets import Button, Layout, HBox, VBox

In [ ]:
app = JupyterFrontEnd()

In [ ]:
open_button = Button(description='Open', button_style='success', icon='folder')
expand_button = Button(description='Expand', button_style='info', icon='chevron-down')
collapse_button = Button(description='Collapse', button_style='info', icon='chevron-right')
hbox = HBox([
    open_button,
    expand_button,
    collapse_button
], layout=Layout(overflow='unset'))
hbox

In [ ]:
def expand_tree(tree, expand=True):
    for node in tree.nodes:
        node.opened = expand

def on_expand_click(b):
    expand_tree(file_tree)

        
def on_collapse_click(b):
    expand_tree(file_tree, False)

expand_button.on_click(on_expand_click)
collapse_button.on_click(on_collapse_click)

In [ ]:
panel = Panel(children=[hbox, file_tree])
panel.title.label = 'Tree Viewer'
panel.title.icon_class = 'jp-FolderIcon'
panel.layout = Layout(overflow='auto')

In [ ]:
app.shell.add(panel, 'left', {'rank': 10000})

In [ ]:
file_tree.nodes[9].opened